In [ ]:
%load_ext autoreload

%autoreload 2

# Exploration du dataset
- Aperçu général: colonnes, valeurs manquantes, duplicats, types de données
- Vérification et nettoyage minimal (si nécéssaire)

In [ ]:
# |exporti
import streamlit as st
import pandas as pd
from streamlit_jupyter import StreamlitPatcher, tqdm

StreamlitPatcher().jupyter()

st.title("Analyse des contenus Netflix")

st.header("Exploration du dataset")

df = pd.read_csv("data/netflix_titles.csv")

# Afficher les colonnes
st.subheader("Colonnes :")
st.write(", ".join([c.title() for c in df.columns]))

# Compter les valeurs manquantes par colonne
st.subheader("Valeurs manquantes par colonne :")
st.dataframe(df.isna().sum().to_frame("Valeurs manquantes"))

# Afficher le nombre de duplcats
st.subheader("\nNombre de doublons :")
st.write(df.duplicated().sum())

# Afficher les types de données
st.subheader("\nTypes de données :")
st.write(df.dtypes.groupby(df.dtypes).size())

# Nettoyer les données en supprimant les doublons
df_cleaned = df.drop_duplicates()
st.write("\nDonnées nettoyées, doublons supprimés.")

# Analyse des contenus
- Films vs séries : proportions, tendances par année.
- Genres principaux (listed_in) : regroupement, fréquences.
- Répartition géographique (country).
- Casting & réalisateurs : analyse simple (comptages, noms fréquents).

In [ ]:
# |exporti
import streamlit as st
import pandas as pd

from streamlit_jupyter import StreamlitPatcher, tqdm

StreamlitPatcher().jupyter()

st.header("Analyse des contenus")

pd.set_option('display.max_rows', None)
df = pd.read_csv("data/netflix_titles.csv")

# Afficher le nombre total de films et son pourcentage
total_movies = df[df["type"] == "Movie"]
st.subheader("Nombre total de films :")
st.write(len(total_movies), " (", round(len(total_movies) / len(df) * 100, 2), "%)")

# Afficher le nombre total de séries et son pourcentage
total_series = df[df["type"] == "TV Show"]
st.subheader("Nombre total de séries :")
st.write(len(total_series), " (", round(len(total_series) / len(df) * 100, 2), "%)")

# Afficher le nombre de films et séries par année de sortie
st.subheader("Nombre de films et séries par année de sortie :")
pivot_df = df.pivot_table(index='type', columns='release_year', values='show_id', aggfunc='count', fill_value=0)
st.write(pivot_df)

# Afficher le nombre de films et séries par catégorie
st.subheader("Nombre de films et séries par catégorie :")
df_listed_in = df.dropna(subset=['listed_in']).copy()
df_listed_in['listed_in'] = df_listed_in['listed_in'].str.split(', ') # Séparer les catégories
df_exploded = df_listed_in.explode('listed_in') # Exploser les lignes pour chaque catégorie
category_type_counts = df_exploded.groupby(['listed_in', 'type'])['show_id'].count().unstack(fill_value=0) # Grouper et compter
categories = sorted(df_exploded['listed_in'].unique())
selected_value = st.selectbox("Sélectionner une catégorie :", options=categories, width=200) # Sélection de la catégorie
st.dataframe(
    category_type_counts.loc[[selected_value]], column_config={
        "listed_in": "Catégorie",
        "Movie": "Films",
        "TV Show": "Séries",
    }
)

# Afficher le nombre de films et séries par pays
st.subheader("Nombre de films et séries par pays :")
df_countries = df.dropna(subset=['country']).copy()
df_countries['country'] = df_countries['country'].str.split(', ') # Séparer les pays
df_exploded = df_countries.explode('country') # Exploser les lignes pour chaque pays
country_type_counts = df_exploded.groupby(['country', 'type'])['show_id'].count().unstack(fill_value=0) # Grouper et compter le nombre de films et séries par pays
countries = sorted(df_exploded['country'].unique())
selected_value = st.selectbox("Sélectionner un pays :", options=countries, width=200) # Sélection du pays
st.dataframe(
        country_type_counts.loc[[selected_value]], column_config={
            "country": "Pays",
            "Movie": "Films",
            "TV Show": "Séries",
        }
    )

# Afficher les 10 acteurs les plus fréquents
st.subheader("Les acteurs les plus fréquents :")
selected_number = st.number_input("Nombre d'acteurs à afficher :", min_value=1, max_value=50, value=10, key="num_actors", width=200)
df_cast = df.dropna(subset=['cast']).copy()
df_cast['cast'] = df_cast['cast'].str.split(', ') # Séparer les acteurs
df_exploded = df_cast.explode('cast') # Exploser les lignes pour chaque acteur
actor_counts = df_exploded['cast'].value_counts().head(selected_number).to_frame() # Compter les acteurs les plus fréquents
st.dataframe(
    actor_counts, column_config={
        "cast": "Acteur",
        "count": "Nombre d'apparitions",
    }
)

# Afficher les 10 réalisateurs les plus fréquents
st.subheader("Les réalisateurs les plus fréquents :")
selected_number = st.number_input("Nombre de réalisateurs à afficher :", min_value=1, max_value=50, value=10, key="num_directors", width=200)
df_directors = df.dropna(subset=['director']).copy()
df_directors['director'] = df_directors['director'].str.split(', ') # Séparer les réalisateurs
df_exploded = df_directors.explode('director') # Exploser les lignes pour chaque réalisateur
director_counts = df_exploded['director'].value_counts().head(selected_number).to_frame() # Compter les réalisateurs les plus fréquents
st.dataframe(
    director_counts, column_config={
        "director": "Réalisateur",
        "count": "Nombre de réalisations",
    }
)

# Analyse temporelle
- Distribution par année de sortie (release_year).
- Analyse de la colonne date_added.

In [ ]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from streamlit_jupyter import StreamlitPatcher, tqdm

StreamlitPatcher().jupyter()

st.header("Analyse temporelle")

st.subheader("Distribution des contenus par année de sortie")
min_year = int(df["release_year"].min())
max_year = int(df["release_year"].max())
year_min, year_max = st.slider(
    "Filtrer les années de sortie :",
    min_value=min_year,
    max_value=max_year,
    value=(2000, max_year),
)



df_year_filtered = df[df["release_year"].between(year_min, year_max)]
titles_per_year = (
    df_year_filtered
    .groupby("release_year")["show_id"]
    .count()
    .sort_index()
)
st.write("Nombre de titres (films + séries) par année de sortie :")
st.bar_chart(titles_per_year)






type_year_counts = (
    df_year_filtered
    .groupby(["release_year", "type"])["show_id"]
    .count()
    .unstack(fill_value=0)
    .sort_index()
)
st.write("Répartition des films et séries par année de sortie :")
st.area_chart(type_year_counts)





st.subheader("Analyse des dates d'ajout sur Netflix")
df_added = df.dropna(subset=["date_added"]).copy()
df_added["date_added"] = df_added["date_added"].str.strip()
df_added["date_added"] = pd.to_datetime(df_added["date_added"], errors="coerce")
df_added = df_added.dropna(subset=["date_added"])
df_added["year_added"] = df_added["date_added"].dt.year
df_added["month_added"] = df_added["date_added"].dt.month
titles_added_per_year = (
    df_added
    .groupby("year_added")["show_id"]
    .count()
    .sort_index()
)
st.write("Nombre de titres ajoutés sur Netflix par année :")
st.bar_chart(titles_added_per_year)




st.subheader("Zoom sur les ajouts récents")
min_added_year = int(df_added["year_added"].min())
max_added_year = int(df_added["year_added"].max())
default_start_year = max(max_added_year - 5, min_added_year)
recent_min_year = st.slider(
    "Afficher les ajouts à partir de l'année :",
    min_value=min_added_year,
    max_value=max_added_year,
    value=default_start_year,
)
df_recent = df_added[df_added["year_added"] >= recent_min_year]
titles_recent_per_year = (
    df_recent
    .groupby(["year_added", "type"])["show_id"]
    .count()
    .unstack(fill_value=0)
    .sort_index()
)
st.write(f"Nombre de films et séries ajoutés par année depuis {recent_min_year} :")
st.bar_chart(titles_recent_per_year)




st.subheader("Nombre de Films vs Séries")
fig, ax = plt.subplots(figsize=(8, 5))
sns.countplot(data=df, x="type", palette="viridis", ax=ax)
ax.set_title("Nombre de Films vs Séries sur Netflix")
ax.set_xlabel("Type de contenu")
ax.set_ylabel("Nombre de titres")
ax.grid(axis='y', linestyle='--', alpha=0.4)
st.pyplot(fig)





st.subheader("Distribution de la durée des films (boxplot)")
df_movies = df[df["type"] == "Movie"].dropna(subset=["duration"]).copy()
df_movies["duration_min"] = df_movies["duration"].str.replace(" min", "").astype(int)
fig, ax = plt.subplots(figsize=(8, 5))
sns.boxplot(x=df_movies["duration_min"], color="skyblue", ax=ax)
ax.set_title("Boxplot de la durée des films (en minutes)")
ax.set_xlabel("Durée en minutes")
st.pyplot(fig)




st.subheader("Top 5 des catégories les plus populaires")
df_cat = df.dropna(subset=["listed_in"]).copy()
df_cat["listed_in"] = df_cat["listed_in"].str.split(", ")
df_exploded = df_cat.explode("listed_in")
top5_categories = df_exploded["listed_in"].value_counts().head(5)
fig, ax = plt.subplots(figsize=(6, 6))
ax.pie(
    top5_categories.values,
    labels=top5_categories.index,
    autopct="%1.1f%%",
    startangle=90,
    colors=plt.cm.Paired.colors
)
st.pyplot(fig)







# Visualisation
- Représentations obligatoires :
- histogrammes, countplots, boxplots (Seaborn / Matplotlib)
- visualisations interactives (Plotly)
- Choix libres : nuages de mots, diagrammes circulaires, timelines, etc.

# Synthèse
- Résumé des observations principales.
- Tendances remarquables (contenus récents, pays dominants, genres populaires).

In [ ]:
st.header("Synthèse")

st.markdown("""
Le catalogue Netflix analysé montre plusieurs tendances intéressantes.

**Contenus récents :**  
La majorité des titres datent des années récentes, surtout après 2010. Les années 2016 à 2019 sont celles où Netflix a ajouté le plus de nouveaux contenus.

**Pays dominants :**  
Les États-Unis dominent largement le catalogue, suivis par l’Inde. On observe également une croissance notable des productions internationales, signe que Netflix élargit son offre mondiale.

**Duré des films :**  
On observe que la plupart des films ont une durée d’environ 90 minutes, ce qui correspond au format le plus répandu dans l’industrie. La distribution est assez concentrée autour de cette valeur, avec quelques outliers représentant des films très courts ou beaucoup plus longs.
            
**Genres populaires :**  
Les catégories les plus représentées sont *International Movies*, *Dramas*, *Comedies* et les contenus pour enfants. Ce sont les genres que Netflix semble pousser le plus.

**Production :**  
Le nombre de productions a explosé à partir de 2015. On voit clairement que le catalogue atteint un sommet autour de 2019. Rien n’est comparable avec les années 2000 et les années précédentes, où le nombre de titres ajoutés était beaucoup plus faible. Cela montre bien l’expansion massive de Netflix durant cette période.

**Conclusion :**  
Netflix propose un catalogue majoritairement récent, dominé par les productions américaines.
""")

# Exportation du notebook 

In [ ]:
from nbdev.export import nb_export
nb_export('project.ipynb', lib_path='./app/', name='netflix_analysis')